# Classifier

### Connection to DB

In [1]:
# Récupération variable enrionnement
import os

MONGODB_URI = os.getenv("MONGODB_URI")

In [2]:
from pymongo import MongoClient

client = MongoClient(MONGODB_URI)

if client is not None:
    print("Connected to DB")
else:
    print("Failed to connect to DB")

Connected to DB


In [14]:
# Base de données
db = client["ai-wedder"]

print("Successfully connected to :", db.name)

Successfully connected to : ai-wedder


### Chargement des datas

In [21]:
# Modèle cible
Review = db["reviews"]

# Récupération des données
reviews = Review.find({})

In [22]:
# Transformation en DF
import pandas as pd

df = pd.DataFrame(reviews)
df

,_id,venue,author,rating,text,date,sentiment,source,createdAt,updatedAt,__v
0,6852d3d45485800971505808,6852d3d474c91954046c1cdb,Estelle,5.0,Monsieur M a surpassé toutes nos attentes lors...,2024-04-11,positive,mariages.net,2025-06-18 14:57:24.287,2025-06-18 14:57:24.287,0
1,6852d3d4548580097150580b,6852d3d474c91954046c1cdb,Patricia,5.0,Nous recommandons vivement les services de Mon...,2025-05-25,positive,mariages.net,2025-06-18 14:57:24.304,2025-06-18 14:57:24.304,0
2,6852d3d4548580097150580e,6852d3d474c91954046c1cdb,Julie,4.8,Un grand merci à Camille et à toute l’équipe d...,2024-12-10,positive,mariages.net,2025-06-18 14:57:24.321,2025-06-18 14:57:24.321,0
3,6852d3d45485800971505811,6852d3d474c91954046c1cdb,Amandine,5.0,"L’équipe était très professionnel, souriante e...",2024-11-09,positive,mariages.net,2025-06-18 14:57:24.336,2025-06-18 14:57:24.336,0
4,6852d3d45485800971505814,6852d3d474c91954046c1cdb,Mathilde,5.0,"Accompagnement en amont, prestation le jour J,...",2025-02-02,positive,mariages.net,2025-06-18 14:57:24.350,2025-06-18 14:57:24.350,0
...,...,...,...,...,...,...,...,...,...,...,...
4406,6852dd207455fa4953903d41,6852dd2074c91954046c1d84,Cocolino,5.0,Une très bonne alternative pour le retour de n...,2016-09-28,positive,mariages.net,2025-06-18 15:37:04.470,2025-06-18 15:37:04.470,0
4407,6852dd207455fa4953903d44,6852dd2074c91954046c1d84,Nicolas,5.0,M. Cousin est un prestataire très chaleureux !...,2016-09-26,positive,mariages.net,2025-06-18 15:37:04.499,2025-06-18 15:37:04.499,0
4408,6852dd207455fa4953903d47,6852dd2074c91954046c1d84,Priscille,5.0,Très apprécié par les invites pour notre lende...,2016-05-31,positive,mariages.net,2025-06-18 15:37:04.519,2025-06-18 15:37:04.519,0
4409,6852dd207455fa4953903d4b,6852dd2074c91954046c1d85,Marie,4.8,"Préparation réalisée sans accros, la qualité d...",2024-07-01,positive,mariages.net,2025-06-18 15:37:04.556,2025-06-18 15:37:04.556,0
